In [1]:
import os
os.chdir("../")
%pwd

'c:\\Users\\supre\\PycharmProjects\\RenalHealth-AI'

In [2]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/spraharaj-projects/RenalHealth-AI.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "spraharaj-projects"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "f875195564dc9cd83db070e2edd56ad426bd2bb8"

In [3]:
from torch import (
  load as torch_load
)

In [4]:
model = torch_load("artifacts/prepare_base_model/base_model_updated.pt")
model_state_dict = torch_load("artifacts/training/model.pt")


In [5]:
model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [6]:
from torchsummary import summary

summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [7]:
from dataclasses import dataclass
from pathlib import Path

In [27]:
@dataclass(frozen=True)
class EvaluationConfig:
    model_path: Path
    model_state_dict_path: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int
    params_learning_rate: int

In [28]:
from cnn_classifier.constants import *
from cnn_classifier.utils.common import read_yaml, create_directories, save_json

In [29]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            model_path="artifacts/prepare_base_model/base_model_updated.pt",
            model_state_dict_path="artifacts/training/model.pt",
            training_data="artifacts/data_ingestion/kidney-dataset",
            mlflow_uri="https://dagshub.com/spraharaj-projects/RenalHealth-AI.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
        )
        
        return eval_config

In [30]:
from torchvision.transforms import v2
from torchvision.datasets import ImageFolder
from torch import (
    device as torch_device,
    cuda as torch_cuda,
    max as torch_max,
    float32 as torch_float32,
    no_grad,
    nn,
)
from torch.optim import SGD
from torch.utils.data import DataLoader, SubsetRandomSampler
import mlflow
from urllib.parse import urlparse

In [33]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        self.device = torch_device(
            'cuda' if torch_cuda.is_available() else 'cpu'
        )

    def _valid_generator(self):
        valid_transform = v2.Compose([
            v2.Resize(self.config.params_image_size[:-1]),
            v2.ToImage(),
            v2.ToDtype(torch_float32, scale=True),
            v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        dataset = ImageFolder(root=self.config.training_data)
        
        num_train = len(dataset)
        indices = list(range(num_train))
        split = int(0.3 * num_train)
        _, valid_indices = indices[split:], indices[:split]

        valid_sampler = SubsetRandomSampler(valid_indices)

        self.valid_loader = DataLoader(
            dataset,
            batch_size=self.config.params_batch_size,
            sampler=valid_sampler,
        )

        self.valid_loader.dataset.transform = valid_transform
    
    @staticmethod
    def load_model(model_path: Path, model_state_dict_path: Path):
        model = torch_load(model_path)
        model_state_dict = torch_load(model_state_dict_path)
        model.load_state_dict(model_state_dict)
        return model

    def validate(self):
        self.model.eval()
        
        valid_running_loss = 0.0
        valid_correct = 0
        valid_total = 0

        with no_grad():
            for data in self.valid_loader:
                inputs, labels = data[0].to(self.device), data[1].to(self.device)
                outputs = self.model(inputs)
                
                loss = self.criterion(outputs, labels)
                
                valid_running_loss += loss.item()

                _, predicted = torch_max(outputs.data, 1)
                valid_total += labels.size(0)
                valid_correct += (predicted == labels).sum().item()

        return [
            valid_running_loss / len(self.valid_loader.dataset),
            (valid_correct / valid_total) * 100,
        ]
    
    def compile(self):
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = SGD(
            self.model.parameters(),
            lr=self.config.params_learning_rate
        )

    def evaluation(self):
        self.model = self.load_model(
            self.config.model_path,
            self.config.model_state_dict_path)
        self._valid_generator()
        self.compile()
        self.score = self.validate()
        self.save_score()
    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)
    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {
                    "loss": self.score[0],
                    "accuracy": self.score[1],
                }
            )
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Register, which depends
                # on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.pytorch.log_model(
                    self.model,
                    "model",
                    registered_model_name="VGG16Model",
                )
            else:
                mlflow.pytorch.log_model(self.model, "model")

In [34]:
try:
    config = ConfigurationManager()
    evaluation_config = config.get_evaluation_config()
    evaluation = Evaluation(evaluation_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e


[2024-04-03 14:46:29,256: INFO: common: yaml file: config\config.yml loaded successfully]
[2024-04-03 14:46:29,258: INFO: common: yaml file: params.yml loaded successfully]
[2024-04-03 14:46:29,258: INFO: common: created directory at: artifacts]


2024/04/03 14:47:02 WARNING mlflow.utils.requirements_utils: Found torch version (2.2.2+cu121) contains a local version label (+cu121). MLflow logged a pip requirement for this package as 'torch==2.2.2' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2024/04/03 14:47:12 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.17.2+cu121) contains a local version label (+cu121). MLflow logged a pip requirement for this package as 'torchvision==0.17.2' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


[2024-04-03 14:52:56,056: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=5, redirect=5, status=5)) after connection broken by 'SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:2406)')': /spraharaj-projects/RenalHealth-AI.mlflow/api/2.0/mlflow-artifacts/artifacts/52f18178bd5749e48d0e6cc9edc9d298/c01590de703340b5a3bb9c78fffe7ab6/artifacts/model/data/model.pth]


Successfully registered model 'VGG16Model'.
2024/04/03 15:00:25 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.


In [1]:
import torch

In [2]:
a = torch.randn(25000, 25000).to('cuda')

In [7]:
b = torch.randn(25000, 25000).to('cuda')

In [8]:
c = a * b